In [1]:
from openai import OpenAI
import io
import os
from PIL import Image
import base64
import json
import ast
import re
client = OpenAI(api_key = API_KEY)

@staticmethod
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [2]:
from pdfminer.high_level import extract_text

def convert_doc_to_images(path):
    images = convert_from_path(path)
    return images

def extract_text_from_doc(path):
    text = extract_text(path)
    page_text = []
    return text

def extract_invoice_data(txt, acc_numers, inn_numbers):
    system_prompt = f"""
You are a data extraction tool that extracts invoice data from Russian-language PDF files.
   
1. In the invoice, you need to allocate the 'account number' of the buyer and the seller. In the document, the word 'account' will be given in Russian and can be abbreviated, for example, "расч.сч." or "р/с" or "Р/Сч", etc.
2. There are the following candidates for 'account number' in the text: {acc_numers}. Check if any of them belong to the buyer or seller. Ignore the account numbers that belong to the bank.
3. There are the following candidates for 'taxpayer number' in the text: {inn_numbers}. Check if any of them belong to the buyer or seller. Ignore the taxpayer numbers that belong to the bank.
4. The buyer can be named in different ways in the documents, for example, "покупатель", "грузополучатель", "плательщик" or "получатель", etc. The seller can also be named in different ways, for example, "продавец", or "поставщик", etc.
5. Please extract the 'account number' and 'taxpayer number' of buyer and seller in this invoice and then output into JSON, keeping the values in the original language.
6. If a row is blank denote missing fields with "null" values.    
7. Don't interpolate or make up data.
    """
    # print(system_prompt)  
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": f"Here is the text of the invoice: \n{txt}"
            }
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content

def extract_data(text):
    pattern_acc = r'\b\d{20}\b'
    pattern_inn = r'\b\d{10}\b'
    matches_acc = re.findall(pattern_acc, text)
    matches_inn = re.findall(pattern_inn, text)
    acc_numbers = [m for m in matches_acc if m.startswith('4')]
    inns = [m for m in matches_inn]
    res = extract_invoice_data(text, acc_numbers, inns)
    print(ast.literal_eval(res))
    return ast.literal_eval(res)


In [3]:
path = 'data/OCR'

pdf_files = [fnm for fnm in os.listdir(path) if 'pdf' in fnm.lower()]
pred = dict()
for fnm in pdf_files:
    text = extract_text_from_doc(os.path.join(path, fnm))
    # print(fnm)
    pred[fnm] = extract_data(text)
    # break

{'buyer': {'account_number': 'null', 'taxpayer_number': '9705186470'}, 'seller': {'account_number': '40702 810 3 40000032984', 'taxpayer_number': '5003126762'}}
{'buyer': {'account_number': '40702810702300004449', 'taxpayer_number': '7716708608'}, 'seller': {'account_number': '40702810214000000054', 'taxpayer_number': '5027096299'}}
{'buyer': {'account_number': 'null', 'taxpayer_number': '7715984348'}, 'seller': {'account_number': '40702810601300013874', 'taxpayer_number': '7703068485'}}
{'buyer': {'account_number': 'null', 'taxpayer_number': '5001111182'}, 'seller': {'account_number': '40702810601300013874', 'taxpayer_number': '7703068485'}}
{'buyer': {'account_number': 'null', 'taxpayer_number': 'null'}, 'seller': {'account_number': 'null', 'taxpayer_number': 'null'}}
{'buyer': {'account_number': 'null', 'taxpayer_number': '4705038787'}, 'seller': {'account_number': '40702810606000101791', 'taxpayer_number': '7804460240'}}
{'buyer': {'account_number': 'null', 'taxpayer_number': '7810

In [6]:
print(text)

Универсальный
передаточный
документ

Статус:  

1 

1 – счет-фактура
и передаточный
документ (акт)
2 – передаточный
документ (акт)

Счет-фактура № 4318 от 06 июля 2024 г.
Исправление № - от -

Продавец  
Адрес 
ИНН/КПП продавца 

Общество с ограниченной ответственностью "АДАМАНТ СПб"
192019, г. Санкт-Петербург, Глухоозёрское ш, д. 12, лит.А, пом.8-Н, офис 20
7806531377/781101001

Грузоотправитель и его адрес:

он же

Грузополучатель и его адрес: 

ООО "ПЕТРОПОЛИС", 196135, г. Санкт-Петербург, Типанова ул, д. 23, корп.2,стр.1, пом.10Н ч.п.6

К платежно-расчетному документу

-

Документ об отгрузке: 

Покупатель  
Адрес 
ИНН/КПП покупателя 

№ п/п 1-2 № 4318 от 06.07.2024

ООО "ПЕТРОПОЛИС"
196135, г. Санкт-Петербург, Типанова ул, д. 23, корп.2,стр.1, пом.10Н ч.п.6
7805655443/781001001

Валюта: наименование, код  

Российский рубль, код - 643

Идентификатор государственного контракта, договора (соглашения) (при наличии)

Приложение N 1 
к постановлению Правительства Российской Федерации 


In [24]:
true = {
    "1_(12).pdf": 
    {'buyer': {'account_number': '40702810702300004449', 'taxpayer_number': '7716708608'}, 
                   'seller': {'account_number': '40702810214000000054', 'taxpayer_number': '5027096299'}},
    "Счет_на_оплату_№_МС000005503_от_17_07_2024_2_xls.pdf": 
    {'buyer': {'account_number': 'null', 'taxpayer_number': '7719897463'},'seller': {'account_number': '40702810438000419723', 'taxpayer_number': '9731123974'}},
    "Счет_фактура_и_УПД_06_07_24_№_4318_29_460_00_в_т_ч.pdf":
    {'buyer': {'account_number': 'null', 'taxpayer_number': '7805655443'},'seller': {'account_number': 'null', 'taxpayer_number': '7806531377'}},
    "Счет_на_оплату_№_ЦБ-7718_от_17.07.2024.pdf": 
    {'buyer': {'account_number': 'null','taxpayer_number': '5009133727'}, 'seller': {'account_number': '40702810400000326167','taxpayer_number': '5027216461'}}}

In [4]:
pred

{'1574-1_АРКО_6508.pdf': {'buyer': {'account_number': 'null',
   'taxpayer_number': '9705186470'},
  'seller': {'account_number': '40702 810 3 40000032984',
   'taxpayer_number': '5003126762'}},
 '1_(12).pdf': {'buyer': {'account_number': '40702810702300004449',
   'taxpayer_number': '7716708608'},
  'seller': {'account_number': '40702810214000000054',
   'taxpayer_number': '5027096299'}},
 '2917486.pdf': {'buyer': {'account_number': 'null',
   'taxpayer_number': '7715984348'},
  'seller': {'account_number': '40702810601300013874',
   'taxpayer_number': '7703068485'}},
 '2917487.pdf': {'buyer': {'account_number': 'null',
   'taxpayer_number': '5001111182'},
  'seller': {'account_number': '40702810601300013874',
   'taxpayer_number': '7703068485'}},
 '526_23754_202407171503_2794763_spec.pdf': {'buyer': {'account_number': 'null',
   'taxpayer_number': 'null'},
  'seller': {'account_number': 'null', 'taxpayer_number': 'null'}},
 'ИМ9840.pdf': {'buyer': {'account_number': 'null',
   'taxpa

In [27]:
for fnm, tr_data in true.items():
    print(fnm)
    pr_data = pred[fnm]
    for subj in ['buyer','seller']:
        for k, v in tr_data[subj].items():
            if v is not None:
                if v!=pr_data[subj][k]:
                    print(subj, k, v, pr_data[subj][k])
    print("")


1_(12).pdf

Счет_на_оплату_№_МС000005503_от_17_07_2024_2_xls.pdf

Счет_фактура_и_УПД_06_07_24_№_4318_29_460_00_в_т_ч.pdf

Счет_на_оплату_№_ЦБ-7718_от_17.07.2024.pdf



In [6]:
text = extract_text_from_doc(os.path.join('data/OCR', '2917487.pdf'))
extract_data(text)

{'buyer': {'account_number': 'null', 'taxpayer_number': '5001111182'}, 'seller': {'account_number': '40702810601300013874', 'taxpayer_number': '7703068485'}}


{'buyer': {'account_number': 'null', 'taxpayer_number': '5001111182'},
 'seller': {'account_number': '40702810601300013874',
  'taxpayer_number': '7703068485'}}

## Excel files

In [33]:
import pandas as pd
df = pd.read_excel("data/OCR/2917486.xls")

In [44]:
import xlrd
book = xlrd.open_workbook("data/OCR/2917486.xls")
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))
sh = book.sheet_by_index(0)
print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))
print("Cell D30 is {0}".format(sh.cell_value(rowx=29, colx=3)))
for rx in range(sh.nrows):
    print(sh.row(rx))

The number of worksheets is 1
Worksheet name(s): ['Sheet1']
Sheet1 66 35
Cell D30 is 
[empty:'', text:'В наличии сварочная проволока и электроды!', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'']
[empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', text:'РЕКВИЗИТЫ ПРОДАВЦА:', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', text:'*INV2917486N*', empty:'', empty:'', empty:'']
[empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', empty:'', 

In [46]:
x = sh.row(1)

In [1]:
import win32com.client
import pathlib


o = win32com.client.Dispatch("Excel.Application")
o.Visible = False
wb_path = pathlib.Path('data/OCR/2917486.xls')
wb = o.Workbooks.Open(wb_path.absolute())

path_to_pdf = pathlib.Path('data/OCR/2917486.pdf')
print_area = 'A1:G50'

ws = wb.Worksheets[0]

ws.PageSetup.Zoom = False
ws.PageSetup.FitToPagesTall = 1
ws.PageSetup.FitToPagesWide = 1
ws.PageSetup.PrintArea = print_area

ws.ExportAsFixedFormat(0, '11.pdf')